In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Importing the dataset
df = pd.read_csv('sentiment.tsv', delimiter = '\t', header=None,names = ["Sentiment", "SentimentText"])
yes_no_cols = ["Sentiment"]
df[yes_no_cols] = df[yes_no_cols] == 'pos'
df = df[df['SentimentText'].isnull() == False]
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df.head(5)

,Sentiment,SentimentText
0,False,"@jamielewislewis i cant believe it, it really ..."
1,True,having a vodka tonic and looking forward to go...
2,True,@ddlovatofans1neg1 Could you follow me please....
3,True,@jordanknight for once.................. PLEAS...
4,False,Had a dream about a walk in fast food resturau...


In [2]:
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

def tokenize(tweet):    
    tweet = (tweet.lower())
    tokens = tokenizer.tokenize(tweet)
    tokens = filter(lambda t: not t.startswith('@'), tokens)
    tokens = filter(lambda t: not t.startswith('#'), tokens)
    tokens = filter(lambda t: not t.startswith('http'), tokens)
    tokens = list(tokens)
    return tokens

In [3]:
# Cleaning the texts
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import LancasterStemmer

stopset = set(stopwords.words('english')) - set(('over', 'under', 'below', 'more', 'most', 'no', 'not', 'only', 'such', 'few', 'so', 'too', 'very', 'just', 'any', 'once'))

[nltk_data] Downloading package stopwords to /opt/pynb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /opt/pynb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
def postprocess(data, n=1000000):    
    data = data.head(n)
    
    data['tokens'] = data['SentimentText'].map(tokenize)  
#     data.reset_index(inplace=True)
#     data.drop('index', inplace=True, axis=1)
    return data

data = postprocess(df)

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:

def word_split(data):    
    data_new = []
    for word in data:
        word_filter = [i.lower() for i in word.split()]
        data_new.append(word_filter)
    return data_new
 
def word_split_sentiment(data):
    data_new = []
    for (word, sentiment) in data:
        word_filter = [i.lower() for i in word.split()]
        data_new.append((word_filter, sentiment))
    return data_new
    
def word_feats(words):    
    return dict([(word, True) for word in words])
 
stopset = set(stopwords.words('english')) - set(('over', 'under', 'below', 'more', 'most', 'no', 'not', 'only', 'such', 'few', 'so', 'too', 'very', 'just', 'any', 'once'))
     


In [6]:
import collections
import nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier, MaxentClassifier, SklearnClassifier
import csv
from sklearn import cross_validation
from sklearn.svm import LinearSVC, SVC
import random
from nltk.corpus import stopwords
import itertools
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

def stopword_filtered_word_feats(words):
    return dict([(word, True) for word in words if word not in stopset])
 
def bigram_word_feats(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    """
    print words
    for ngram in itertools.chain(words, bigrams): 
        if ngram not in stopset: 
            print ngram
    exit()
    """    
    return dict([(ngram, True) for ngram in itertools.chain(words, bigrams)])
    
def bigram_word_feats_stopwords(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    """
    print words
    for ngram in itertools.chain(words, bigrams): 
        if ngram not in stopset: 
            print ngram
    exit()
    """    
    return dict([(ngram, True) for ngram in itertools.chain(words, bigrams) if ngram not in stopset])


/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
trainfeats = [(word_feats(f), 'neg') for f in word_split(df['SentimentText'])]

In [28]:
for i in range(0, len(df)):
    jpos = 0
    jneg = 0
    dfpos = []
    dfneg = []
    if df['Sentiment'][i]:
        dfpos.append(df['SentimentText'][i])
    else:
        dfneg.append(df['SentimentText'][i])     
        

[]
['having a vodka tonic and looking forward to going to Saddle Ranch at the Westgate with the husband, sans kiddos ']
['@ddlovatofans1neg1 Could you follow me please.I would really appreciate it. ']
['@jordanknight for once.................. PLEASE TELL US WHY u were thinking of this person?!? ']
[]
['@TroyBrownBBNews Yes... For a &quot;friend&quot; ']
['Packing for work expierance ']
[]
[]
[]
['@BritRuxpin. I say answering with no pants. ']
[]
[]
[]
[]
[]
[]
[]
[]
['&quot;so we keep waitin, waitin , waitin on the world to change&quot; - john mayer ']
[]
[]
[]
['@ms_cornwall night you, may your dreams be wonderful ']
['Star Wars: Old Republic: http://digg.com/d1sgupos and it maybe coming to the PS3 and eggbox: http://digg.com/u1poscY2 ']
['You could be my punk rock princess ']
[]
['19 more sleeps untill Take That ahhhhhhhhhhhhhhh ']
['aaaw! my moms crying. She loved the mixed tape i gave her ']
[]
["Morning y'all. Looking forward to a nice BBQ today. Can everyone follow @RebeccaMaste

["@howtobehuge you're 275 pounds of muscle? wow...hawt!  how are you doing today, dustin?"]
["@JeffWalker Let's see, we have pos bathrooms in the house, so, pos books going at all times. "]
[]
["Morning, having some friends over today. I hope we dont get attacked by the You-know-who's. I've been out of their site for a while. "]
["off to my auntie's to chill in the sun. "]
[]
['@zappos Congrats, happy 1negth anniversary ']
['soooooo tired. love days like this ']
["@BBJlo no clue! Here's to hoping tho! "]
[]
[]
["Taking our lab babies to the lake  it's a gorgeous day!"]
['Made it to DC after nearly missing the bus. I ran so fast to make it I tripped &amp; ate pavement. Made some strangers on the street laugh ']
["He told me he's not &quot;crazy&quot;...just &quot;creatively retarded&quot; (lol) or something to that effect...shouts out to my Baloney Face  who else  ..."]
[]
[]
["@futuredirected  It's absolutely your right!    (and a well-deserved one! But at least I'm upfront about my em

[]
[]
[]
["@KimKardashian i'm SOOO excitedd "]
[]
["@shayneTward i'll join you for a pint Shayne "]
["@Animadi You're welcome and thank you. "]
[]
[]
['@zealandsmom Thx for that! ']
[]
["@toyhorses Exellent, I'll be over about 1:3neg ish for pool antics and shots of absinthe "]
['watching im a celebrity.....get me out of here.  pretty crazy']
['@davynathan of course we r!! BHs r nothing if not loyal!! Good knight! ']
['@MrPeterAndre no problem Pete hope you are ok ']
['@successfool You know it PIC!  I believe in you big time!']
['Hanging out at home ']
[]
['@Andreaheartscgh @Catherine_andy Remember to youtube Zodiac show crawl thru fire  so hot man']
[]
['@khallwalker experience shows that sending the kids/partner to get some is the more successful alternative ']
['bye bye twitter world ']
[]
[]
["just got done editing pix from yesterday's #gbtu. Will post them tomorrow night and send out a link! "]
[]
["@tinhet Hey you      It surprisingly didn't hurt that much..as for your foot...I d

In [29]:
print(dfpos)

['@queith asi es! ']


In [ ]:
n=1000000
x_train, x_test, y_train, y_test = train_test_split(np.array(data.head(n).tokens),
                                                    np.array(data.head(n).Sentiment), test_size=0.2)

In [ ]:
import collections
import nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews

def evaluate_classifier(featx):
    

    classifier = NaiveBayesClassifier.train(x_train,y_train)
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)
 
    for i, (feats, label) in enumerate(testfeats):
            refsets[label].add(i)
            observed = classifier.classify(feats)
            testsets[observed].add(i)
     
    classifier.show_most_informative_features()

In [ ]:
x_train

In [ ]:
def word_feats(words):
    return dict([(word, True) for word in words])
 
evaluate_classifier(word_feats)


In [ ]:
tokens

In [ ]:
 tokens = bag_of_bigrams_words(tokens)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3),token_pattern=r'\b\w+\b', min_df=5,max_features = 1500)
X = vectorizer.fit_transform(corpus).toarray()

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(max_features = 1500)
# X = cv.fit_transform(corpus).toarray()

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
# features = vectorizer.fit_transform(corpus)
# X = features.toarray()

In [ ]:
i = 1000
j = 10
vectorizer.get_feature_names( )[i:i+10]

In [ ]:
feature_names = vectorizer.get_feature_names( )
print(feature_names[::10])

In [ ]:
# prop.table(table(df$sentiment))
yes_no_cols = ["sentiment"]
df[yes_no_cols] = df[yes_no_cols] == 'pos'
y = df.iloc[:, 0].values

In [ ]:
from sklearn.metrics import accuracy_score as accuracy
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

# from sklearn.naive_bayes import GaussianNB
# from sklearn import linear_model
# from sklearn import tree
# from sklearn import svm
# from sklearn import ensemble
# from sklearn import neighbors

kf = KFold(len(y),n_folds=10,shuffle=True)
kf2 = StratifiedKFold(y,n_folds=10,shuffle=True)


results = cross_val_score(LogisticRegression(), X = X, y = y, scoring = "roc_auc", cv = kf)
print("LogisticRegression Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

results = cross_val_score(SGDClassifier(loss='log', penalty='l1'), X = X, y = y, scoring = "roc_auc", cv = kf)
print("SGDClassifier Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

# results = cross_val_score(GaussianNB(), X = X, y = y, scoring = "roc_auc", cv = kf)
# print("GaussianNB Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))
# results = cross_val_score(ensemble.RandomForestClassifier(), X = X, y = y, scoring = "roc_auc", cv = kf)
# print("RandomForest Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))
# results = cross_val_score( ensemble.GradientBoostingClassifier(), X = X, y = y,scoring = "roc_auc", cv = kf)
# print("GradientBoosting Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, scoring = "roc_auc", cv=10)
grid.fit(X, y)
print("Best cross-validation score: ", grid.best_score_)
print("Best parameters: ", grid.best_params_)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
"tfidfvectorizer__ngram_range": [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]}
grid = GridSearchCV(pipe, param_grid, scoring = "roc_auc", cv=10)
grid.fit(corpus, y)
print("Best cross-validation score: ", grid.best_score_)
grid.best_params_

In [ ]:
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.layers.core import Activation, Dense, Dropout
from keras.models import Sequential

def DefineModel1():    
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=615))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = DefineModel1()     
model.fit(X, y, epochs=50, batch_size=32, verbose=2)

In [ ]:
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score as accuracy
from keras.wrappers.scikit_learn import KerasClassifier
import numpy

# create model
model = KerasClassifier(build_fn=DefineModel1, epochs=50, batch_size=32, verbose=0)
# model = KerasClassifier(build_fn=DefineModel1, epochs=50, batch_size=32)
# evaluate using 10-fold cross validation
# kf = KFold(len(y),n_folds=10,shuffle=True)
kf2 = StratifiedKFold(y,n_folds=10,shuffle=True, random_state=7)
results = cross_val_score(model, X, y,scoring = "roc_auc", cv=kf2)
print("Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

# Summary of roc_auc:
KerasClassifier: Accuracy: 0.713 (0.038)
LogisticRegression Accuracy: 0.751 (0.017)
SGDClassifier Accuracy: 0.723 (0.018)